In [15]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from statistics import mean
import os
from flask import Flask
from flask import request

In [16]:
# folder path
dir_path = r'..\Python and Data Science\WindDataSet'
selection_string = "T111_4_"
# list to store files
res = []

# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        res.append(path)
filteredDir_path = [file_name for file_name in res if selection_string in file_name]
len(filteredDir_path)

6

In [17]:
# To read all data from mat file
pc_list = {}
try:
    checkfile = open(".\Mean pc Data " + selection_string + ".txt",'r')
    checkfile.close()
except:
    for file_name in filteredDir_path:
        data_set = loadmat('./WindDataSet/' + file_name)
        for time, value in enumerate(data_set['Wind_pressure_coefficients']):
            winddirection = data_set["Wind_direction_angle"][0][0]
            for point, pressure_coefficient in enumerate(value):
                x = data_set['Location_of_measured_points'][0][point]
                y = data_set['Location_of_measured_points'][1][point]
                side = data_set['Location_of_measured_points'][3][point]
                roundedX = round(x*100)
                roundedY = round(y*100)
                uniqueKey = "A" + str(roundedX) + "_" + str(roundedY) + "_" + str(round(side)) + "_" + str(round(winddirection))
                if uniqueKey in pc_list:
                    pc_list[uniqueKey].append(pressure_coefficient)
                else:
                    pc_list[uniqueKey] = []
    file = open(".\Mean pc Data " + selection_string + ".txt",'w')
    print(len(pc_list.keys()))
    for key in pc_list.keys():
        x = key.split("_")[0][1:]
        y = key.split("_")[1]
        side = key.split("_")[2]
        wind_direction = key.split("_")[3]
        mean_pc = mean(pc_list[key])
        file.write(x +"," + y +"," + side +"," + wind_direction + "," + str(mean_pc) + "\n" )
    file.close()

In [18]:
pc_mean_list = []
file = open(".\Mean pc Data " + selection_string + ".txt",'r')
for line in file:
    all_values = line.split(",")
    entry = {"x": int(all_values[0])/100, "y": int(all_values[1])/100, "side": all_values[2], "wind_direction": all_values[3], "pc": all_values[4].split("\n")[0]}
    pc_mean_list.append(entry)
file.close()
pc_mean_list[0]

{'x': 0.01, 'y': 0.09, 'side': '1', 'wind_direction': '0', 'pc': '0.5117052'}

In [19]:
pc_df = pd.DataFrame(pc_mean_list)

In [20]:
normalizer = MinMaxScaler()
X = pc_df.drop("pc", axis=1)
y = pc_df["pc"]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)
X_train[["x","y","side","wind_direction"]] = normalizer.fit_transform(X_train[["x","y","side","wind_direction"]])
X_test[["x","y","side","wind_direction"]] = normalizer.transform(X_test[["x","y","side","wind_direction"]])
X_train

,x,y,side,wind_direction
381,0.052632,0.00,0.000000,0.6
263,0.157895,0.25,0.000000,0.4
296,0.842105,0.00,1.000000,0.4
236,0.842105,0.75,1.000000,0.4
173,0.684211,0.25,0.666667,0.2
...,...,...,...,...
572,0.631579,0.25,0.666667,1.0
34,0.736842,0.75,0.666667,0.0
186,0.315789,0.00,0.333333,0.2
297,0.894737,0.00,1.000000,0.4


In [21]:
pc_df.describe()

,x,y
count,600.000000,600.000000
mean,0.200000,0.050000
std,0.115422,0.028308
min,0.010000,0.010000
25%,0.105000,0.030000
50%,0.200000,0.050000
75%,0.295000,0.070000
max,0.390000,0.090000


In [29]:
app = Flask(__name__)

@app.route('/')
def predict_pc():
    
    x = request.args.get('x')
    y = request.args.get('y')
    side = request.args.get('side')
    winddirection = request.args.get('winddirection')
    modeltype = request.args.get('model')
    testentry= [[int(x),int(y),int(side),int(winddirection)]]


    model_list = ["linear", "logistic","svr_linear","svr_rbf"]

    def model_predict(modeltype):
        prediction_model = ""
        match modeltype:
            # Linear regression - straight line fitting
            case "linear":
                prediction_model = LinearRegression()
            # Logistic regression - logarithmic fitting
            case "logistic":
                prediction_model = LogisticRegression()
            # Support Vector Regression - Linear
            case "svr_linear":
                prediction_model = SVR(kernel='linear')
            # Support Vector Regression - Rbf
            case "svr_rbf":
                prediction_model = SVR(kernel='rbf')

        if prediction_model == "":
            return "Model Invalid!"
        
        prediction_model.fit(X_train,y_train)
        y_predict = prediction_model.predict(X_test)
        mse = mean_squared_error(y_test, y_predict)
        api_y_predict = prediction_model.predict(testentry)
        response = {"model": modeltype, "predicted_pc": list(api_y_predict)[0], "mean_squared_error": mse}
        return response
    
    result = []
    if modeltype == "all":
        for model_type in model_list:
            result.append(model_predict(model_type))
    else:
        result = model_predict(modeltype)
    return result
    

if __name__ == '__main__':
   app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
c:\Users\chali\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\chali\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\chali\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
c:\Users\chali\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
127.0.0.1 - - [11/May/2024 02:34:52] "GET /?model=all&x=1&y=9&side=1&winddirection=0 HTTP/1.1" 200 -


[[1, 9, 1, 0]]
[[1, 9, 1, 0]]
[[1, 9, 1, 0]]
[[1, 9, 1, 0]]


c:\Users\chali\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\chali\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\chali\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
c:\Users\chali\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
127.0.0.1 - - [11/May/2024 02:34:55] "GET /?model=all&x=1&y=9&side=1&winddirection=0 HTTP/1.1" 200 -


[[1, 9, 1, 0]]
[[1, 9, 1, 0]]
[[1, 9, 1, 0]]
[[1, 9, 1, 0]]
